In [1]:
import xarray as xr
import os
import numpy as np
import pandas as pd
from datetime import datetime

res = 2.8125
local_dir = f"E:\\weatherbench\\{res}deg"

In [69]:
d=xr.open_dataset("E:\\weatherbench\\2.8125deg\\toa_incident_solar_radiation\\toa_incident_solar_radiation_2.8125deg\\toa_incident_solar_radiation_1979_2.8125deg.nc")

In [94]:
lon = 0.0
lat = 
#data = d.query(f"lon=={lon} and lat=={lat}")
data = d.sel(lon=lon, lat=lat).to_dataframe()['tisr']
one_year_daily=np.array([data[i:i+24].mean(axis=0) for i in range(0, len(data), 24)])

In [2]:
def get_all_4_corners():
    d = xr.open_dataset("E:\\weatherbench\\2.8125deg\\2m_temperature\\2m_temperature_2.8125deg\\2m_temperature_1979_2.8125deg.nc")
    lats = [lt.item() for lt in d['lat']]
    lons = [ln.item() for ln in d['lon']]
    output = []
    for i, lat in enumerate(lats):
        for j, lon in enumerate(lons):
            if i == len(lats)-1:
                continue
            if j == len(lons)-1:
                continue
            output.append([(lat, lon), (lat, lons[i+1]), (lats[i+1], lon), (lats[i+1], lons[i+1])])
    return output


def get_daily_value_for_year(feature_info, lat, lon, year):
    feature, df_fld = feature_info
    d = xr.open_dataset(os.path.join(local_dir, feature, f"{feature}_{res}deg", f"{feature}_{year}_{res}deg.nc"))
    data = d.sel(lon=lon, lat=lat).to_dataframe()[df_fld]
    # hourly temperature: each day covers 24 entries. Take the daily average over 24hrs
    one_year_daily=np.array([np.mean(data[i:i+24]) for i in range(0, len(data), 24)])
    return one_year_daily[:365].reshape((1, 365))


def linearize(x):
    linearized = np.array([])
    for chunk in x:
        linearized = np.concatenate((linearized, chunk), axis=0)
    return linearized


def detrend(feature, lat, lon, years, nombor=0):
    """
    Extract the mean feature value for each day in a year over the period between y1 and y2 inclusive and use
    it to do seasonal detrending.
    """
    y1, y2 = years
    data = None
    for y in range(y1, y2+1):
        more_data = get_daily_value_for_year(feature, lat, lon, y)
        if data is None:
            data = more_data
        else:
            data = np.concatenate((data, more_data), axis=0)
#    np.savetxt(f"daily_averages{nombor}.txt", data, delimiter=',')
    # means is an array where mean[0] is the feature value average over the years between y1 and y2 on the 
    # on the first day of a year
    means = np.array([np.array([data[i, j] for i in range(y2-y1+1)]).mean() for j in range(365)])
    detrended = data - means
    return detrended, means, data

def get_autummn_day_indices():
    autumn_1st_day = datetime(1979, 9, 22).timetuple().tm_yday
    autumn_last_day = datetime(1979, 12, 22).timetuple().tm_yday
    return (autumn_1st_day, autumn_last_day)


In [3]:
semua = get_all_4_corners()
semua[4000][0]

(-1.40625, 177.1875)

In [22]:
first_day, last_day = get_autummn_day_indices()
avgs_empat = []
year_start = 1979
year_end = 1982
for pt in semua[4000]:
    detrended, _, _ = detrend(("toa_incident_solar_radiation", 'tisr'), *pt, (year_start, year_end))
    avgs_empat.append([x.mean() for x in detrended[:, first_day:last_day]])


In [35]:
pd_data = pd.DataFrame(data=np.array(avgs_empat).T, columns=['loc0', 'loc1', 'loc2', 'loc3'])
pd_data

,loc0,loc1,loc2,loc3
0,63.202610,-201.821429,-102.199519,-333.901099
1,326.640797,401.210165,611.827953,677.048077
2,-23.581731,75.223901,-154.277816,-68.027473
3,-366.261676,-274.612637,-355.350618,-275.119505


In [11]:
df = pd.read_csv("yearly_winter_anomaly_clusters.csv")

In [33]:
nn

array([[  63.20260989, -201.82142857, -102.19951923, -333.9010989 ],
       [ 326.6407967 ,  401.21016484,  611.8279533 ,  677.04807692],
       [ -23.58173077,   75.2239011 , -154.27781593,  -68.02747253],
       [-366.26167582, -274.61263736, -355.35061813, -275.11950549]])